In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 간단하게 처리하는 방법
# from tensorflow.keras.datasets import imdb

# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

imdb_data = pd.read_csv("IMDB Dataset.csv")
# pos, neg 값을 숫자로 변환
imdb_data['sentiment'] = imdb_data['sentiment'].replace('positive', 1)
# imdb_data['sentiment'].replace('positive', 1, inplace=True)
imdb_data['sentiment'] = imdb_data['sentiment'].replace('negative', 0)
# 정규표현식을 써서, 단어가 아니면 공백으로
imdb_data['review'] = imdb_data['review'].str.replace("[^\w]|br", " ")
# 혹시 공백이 있으면 제거
imdb_data['review'] = imdb_data['review'].replace("", np.nan)
imdb_data['sentiment'] = imdb_data['sentiment'].replace("", np.nan)
# null array 없애는 함수
imdb_data = imdb_data.dropna(how='any')

print("# preprocessing done")

review_train, review_test, y_train, y_test = train_test_split(imdb_data['review'], imdb_data['sentiment'], test_size=0.25, shuffle=False, random_state=23)

print("# split done")

stopwords = ['a', 'an']

X_train = []
for stc in review_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print("# tokenization done")

# 왜 트레인셋만?.. 원하신다면 처음에 test/train 스플릿하기 전에, 전처리하셔서 fit하셔도
# 왜 5000?.. 유의미한 단어 갯수를 생각해보자! 빈도수 1개다 -> 버리자, 빈도수 2개다 -> 버리자
tokenizer = Tokenizer(5000)
# train 을 기준으로 단어마다의 인덱스를 부여
tokenizer.fit_on_texts(X_train)

# 부여된 정수 인덱스로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print("# int_encoding done")

In [ ]:
print(X_train[1])
print(y_train[1])

In [ ]:
# 문장 길이를 맞춰준다
# 임의로 맞추는게 아니고, 데이터셋을 보면서 최대 문장의 길이가 얼마인지 확인하시고 거기에 맞춰서
# imdb일때는 500, 네이버일때는 50으로 맞췄었습니다!
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
print(X_train[1])

In [ ]:
# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개 (imdb) 혹은 20000개 (네이버) 의 단어를 120차원으로 내보내겠다
# 1인자 = 내가 넣을 단어의 갯수 (총 인덱스의 갯수), 2인자 = 출력할 차원 (덴스 벡터의 차원), 3인자 = 문장의 길이
model.add(Embedding(5000, 120))
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, callbacks=[early_stop, model_check])

In [ ]:
# 정확도 측정
# 출력하면 [loss, acc]
print(model.evaluate(X_test, y_test))
# X 값은 전처리, 토큰화, 정수인코딩이 된 상태의 문장이어야
print(model.predict(X))

In [ ]:
!pip install konlpy

In [ ]:
########## 한글 전처리부분입니다 ########

import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split

naver_data = pd.read_table("./drive/My Drive/ratings.txt", encoding='utf-8')
# 단어 아니면 삭제
naver_data['document'] = naver_data['document'].str.replace("[^\w]", " ")
# 혹시 공백이 있으면 null array로
naver_data['document'] = naver_data['document'].replace('', np.nan)
naver_data['label'] = naver_data['label'].replace('', np.nan)
# null array 모두 제거 (공백인 열 모두 제거)
naver_data = naver_data.dropna(how='any')

print("# preproecssing done")

# test/train 스플릿하고
review_train, review_test, y_train, y_test = train_test_split(naver_data['document'], naver_data['label'], test_size=0.25, shuffle=False, random_state=1004)

print('# split done')

# stopwords 지정
stopwords = ['를', '을', '이', '가', '으로', '로', '에', '에서']

# 토큰화 진행
X_train = []
for stc in review_train:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print('# tokenization done')

# X_train 단어들을 토대로 정수 인덱스 설정, 전체 단어 갯수 설정
# 유의미한 단어? 빈도수가 1~2개인 단어 버려도 큰 영향을 끼치지 않을것 -> count함수써서 빈도수 낮은 것들을 버리고, 남은 단어의 갯수들
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(X_train)

# 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')